# Exercise M4.04

In this exercise, we will train an l2-penalized logistic regression classifier and investigate the effect of the parameter `c`.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
penguins = pd.read_csv('data/penguins_classification.csv')
# only keep the Adelie and Chinstrap classes
penguins = penguins.set_index('Species').loc[['Adelie', 'Chinstrap']].reset_index()

In [4]:
penguins.sample(3)

,Species,Culmen Length (mm),Culmen Depth (mm)
94,Adelie,40.8,18.9
178,Chinstrap,49.5,19.0
103,Adelie,37.9,18.6
